In [7]:
import pymongo
from pymongo import MongoClient
client = MongoClient('mongodb://127.0.0.1:27017')
db = client.sample

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

# Features selction by LassoCV and SelectFromModel
test con i 4 differenti insiemi di target
    - numero di likes
    - ranking sul numero di likes
    - numero di likes normalizzato sul numero di utenti del team
    - ranking sul numero di like normalizzato

In [8]:
# caricamento e normalizzazione del dataset (800 teams)

samples = pd.read_csv('../features collection/test data/800 teams multi-target/800_teams_vectors.csv')
samples = samples.drop(samples.columns[0], axis=1)
samples_log = samples.apply(np.log)
samples_log = samples_log.replace(-np.inf, 0)
samples_log = samples_log.fillna(0)
transformer = RobustScaler().fit(samples_log)
samples_scaled = pd.DataFrame(transformer.transform(samples_log), columns=list(samples))
samples_scaled

,team_nusers,team_degree,team_degree_centrality,team_betweenness_centrality,team_closeness_centrality,team_eigenvector_centrality,team_pagerank,team_avg_users_nshot,team_avg_users_in_degree,team_avg_users_out_degree,...,team_avg_users_diversity,team_avg_users_nskills,team_avg_users_nfollowers,team_avg_users_nfollowings,team_avg_users_avg_shots_ranking,team_avg_users_avg_shots_ntags,team_avg_users_avg_shots_ncomments,team_avg_users_avg_shots_nlikes,team_avg_users_avg_shots_nviews,team_avg_users_avg_shots_diversity
0,1.600297,1.098612,-0.946843,0.000000,-0.911758,-5.669961,0.896478,-0.022589,0.372494,0.087823,...,-0.255785,-0.100753,0.796052,0.464537,-0.466405,-0.089010,-0.084854,0.520457,0.991869,-0.270483
1,-1.198978,0.000000,-1.091291,0.000000,-2.014739,0.000000,1.000000,0.847446,-0.033078,0.383258,...,0.181665,0.763306,-0.001545,0.380428,0.629363,-0.374880,-0.215255,-0.110256,-0.525801,0.190521
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.131194,0.542389,-0.478710,...,0.269443,0.218662,0.548351,-0.430610,-1.219078,0.238251,-0.274112,-0.187990,0.068280,-0.688752
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.415768,-0.053850,-0.084184,...,1.072448,0.373785,-0.316596,-0.360429,0.126495,-0.195607,0.402911,0.334893,0.199952,-0.329659
4,1.681175,1.098612,-0.946843,-7.635494,-0.932638,-7.777640,0.987672,-0.333957,0.971774,0.621320,...,0.099372,0.000000,1.087740,1.023630,0.320666,1.215726,1.692073,1.960874,1.682450,0.775236
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.465738,0.152327,1.419375,...,0.247300,0.298164,-0.013027,1.293491,0.401406,0.844045,0.025072,0.009356,-0.015599,0.978625
6,0.756471,0.000000,-1.091291,0.000000,-1.944255,0.000000,0.868197,0.291028,-0.233612,0.289361,...,0.417004,0.177329,-0.003097,0.239184,-0.272082,-0.005249,-0.514653,-0.345409,-0.310491,-0.014725
7,-0.442507,0.000000,-1.091291,0.000000,-2.014739,0.000000,1.000000,0.659131,0.025193,0.263809,...,-2.918224,-0.700547,0.069647,0.171329,-2.236484,0.736134,0.080154,-0.431570,0.001377,-0.406524
8,-0.442507,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.204070,0.086634,1.064418,...,-0.739992,-0.939470,0.627144,1.228632,-2.517670,0.801769,1.147859,1.230841,1.096198,-1.922587
9,2.014455,1.386294,-0.908976,-8.477972,-0.922495,-6.143218,1.248107,0.052064,0.334924,-0.094674,...,-0.189157,-0.180703,0.336076,-0.167343,-0.107768,-0.353054,0.164386,0.420023,0.727849,-0.247494


In [14]:
# caricamento e normalizzazione dela variabile di target (800 teams)
def load_target(path):

    target = pd.read_csv(path)
    target = target.drop(target.columns[0], axis=1)
    target_log = target.apply(np.log)
    target_log = target_log.replace(-np.inf, 0)
    target_log = target_log.fillna(0)
    transformer = RobustScaler().fit(target_log)
    target_scaled = pd.DataFrame(transformer.transform(target_log), columns=['likes'])
    return target_scaled

In [87]:
def features_selection(X, y, labels, thres):

    # We use the base estimator LassoCV since the L1 norm promotes sparsity of features.
    clf = LassoCV(cv=5, alphas=[0.026])

    # Set a minimum threshold of 0.25
    
    sfm = SelectFromModel(clf, threshold=thres)
    sfm.fit(X, y)
    n_features = sfm.transform(X).shape[1]
    
    # print n_features
    

    # Print features scores
    for i in range(0,len(sfm.get_support())):
        if sfm.get_support()[i]:
            print sfm.estimator_.coef_[i], labels[i]
            
    #print(sfm.estimator_.coef_)
            
    # estimator accuracy
    estimator_accuracy(clf, X, y)

In [88]:
def estimator_accuracy(estimator, x, y):
    x_train = x[0:400]
    y_train = y[0:400]
    x_test = x[400:800]
    y_test = y[400:800]
    
    estimator.fit(x_train, y_train)
    pred = estimator.predict(x_test)
    
    print 'Accuracy: ', estimator.score(x_test,y_test)

# N° di likes

In [72]:
thres = 0.1

In [94]:
target_set = load_target('../features collection/test data/800 teams multi-target/800_teams_nlikes_target.csv')
features_selection(samples_scaled, target_set, list(samples), thres)

0.42571910779072536 team_nusers
0.3791416481507632 team_avg_users_nshot
0.23017746895243946 team_avg_users_nfollowers
0.3460518833991648 team_avg_users_avg_shots_nlikes
Accuracy:  0.9794542125007829


# Ranking sul n° di likes

In [90]:
target_set = load_target('../features collection/test data/800 teams multi-target/800_teams_rankLikes_target.csv')
features_selection(samples_scaled, target_set, list(samples), thres)

0.408477237070015 team_nusers
0.17938798426097147 team_avg_users_nshot
0.15194454734368537 team_avg_users_in_degree
0.2977372218406409 team_avg_users_nfollowers
Accuracy:  0.8766194106600644


# N° di likes normalizzato sul n° di utenti del team

In [91]:
target_set = load_target('../features collection/test data/800 teams multi-target/800_teams_nlikesNormalized_target.csv')
features_selection(samples_scaled, target_set, list(samples), thres)

0.4862057702605165 team_avg_users_nshot
0.29579999162370074 team_avg_users_nfollowers
0.4510929506830294 team_avg_users_avg_shots_nlikes
Accuracy:  0.9676884627894882


# Ranking sul n° di like normalizzato

In [92]:
target_set = load_target('../features collection/test data/800 teams multi-target/800_teams_rankLikesNormalized_target.csv')
features_selection(samples_scaled, target_set, list(samples), thres)

0.17963908134239956 team_avg_users_nshot
0.16338553305023104 team_avg_users_in_degree
0.3007716775512595 team_avg_users_nfollowers
Accuracy:  0.7342500723816808


# Accuracy for models with features selected

In [105]:
# caricamento e normalizzazione del dataset (800 teams)
# dataset con features: 
#    team_nusers, team_avg_users_nshot, team_avg_users_nfollowers, team_avg_users_avg_shots_nlikes

samples = pd.read_csv('../features collection/test data/800 teams multi-target/800_teams_vectors_features_selection.csv')
samples = samples.drop(samples.columns[0], axis=1)
samples_log = samples.apply(np.log)
samples_log = samples_log.replace(-np.inf, 0)
samples_log = samples_log.fillna(0)
transformer = RobustScaler().fit(samples_log)
samples_scaled = pd.DataFrame(transformer.transform(samples_log), columns=list(samples))
#samples_scaled

In [106]:
target_set = load_target('../features collection/test data/800 teams multi-target/800_teams_nlikes_target.csv')
x_train = samples_scaled[0:400]
y_train = target_set[0:400]
x_test = samples_scaled[400:800]
y_test = target_set[400:800]

In [107]:
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.svm import SVR

regressor = Lasso(alpha=0.026)
regressor.fit(x_train, y_train)
pred = regressor.predict(x_test)
print 'lasso: ', regressor.score(x_test,y_test)

regressor = SVR(kernel='rbf', C=10, gamma=0.01, epsilon=0.01)
regressor.fit(x_train, y_train)
pred = regressor.predict(x_test)
print 'SVR: ', regressor.score(x_test,y_test)

lasso:  0.9793804027240937
SVR:  0.9842645307253894
